<a href="https://colab.research.google.com/github/Anil-Chalumuri/GenAINotes/blob/main/Pinecone_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pinecone: https://www.pinecone.io/

In [9]:
!pip install langchain
!pip install pinecone-client==2.2.4
!pip install pypdf

In [10]:
!pip install sentence-transformers==2.2.2

In [3]:
!mkdir pdfs

## Extract the Text from the PDF's

In [11]:
from langchain.document_loaders import PyPDFDirectoryLoader

In [12]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [13]:
data

[Document(page_content='CS391R: Robot Learning (Fall 2021)\nYou Only Look Once (YOLO): Unified, Real-Time Object Detection\n1Presenter: Shivang SinghSept 2nd, 2021', metadata={'source': 'pdfs/yolo.pdf', 'page': 0}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy\n“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification', metadata={'source': 'pdfs/yolo.pdf', 'page': 1}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)3Importance of Object Detection for Robotics❖Visual modality is very powerful❖Humans are able to detect objects and do perception using just this modality in real time (not needing radar) ❖If we want responsive robot systems that work in real time (without specialized sensors) almost real time vision based object detection can help greatly\nVi

## Chunkins

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [16]:
len(text_chunks)

29

In [17]:
text_chunks[2]

Document(page_content='CS391R: Robot Learning (Fall 2021)3Importance of Object Detection for Robotics❖Visual modality is very powerful❖Humans are able to detect objects and do perception using just this modality in real time (not needing radar) ❖If we want responsive robot systems that work in real time (without specialized sensors) almost real time vision based object detection can help greatly\nVision based vs LIDAR (self driving)\nTesla Investor Day Presentation', metadata={'source': 'pdfs/yolo.pdf', 'page': 2})

## Embeddings

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [20]:
query_result = embeddings.embed_query("Hello World")

In [21]:
query_result

[-0.03447727486491203,
 0.03102317824959755,
 0.006734995171427727,
 0.026108944788575172,
 -0.039361994713544846,
 -0.16030240058898926,
 0.06692399084568024,
 -0.006441427860409021,
 -0.04745052009820938,
 0.014758813194930553,
 0.07087530195713043,
 0.05552761256694794,
 0.01919334940612316,
 -0.026251301169395447,
 -0.010109584778547287,
 -0.026940491050481796,
 0.0223073810338974,
 -0.022226644679903984,
 -0.1496926248073578,
 -0.017493028193712234,
 0.007676266599446535,
 0.05435226857662201,
 0.0032544408459216356,
 0.031725917011499405,
 -0.08462139219045639,
 -0.0294059906154871,
 0.05159558355808258,
 0.04812406376004219,
 -0.0033148028887808323,
 -0.058279186487197876,
 0.041969265788793564,
 0.02221064269542694,
 0.1281888484954834,
 -0.02233898639678955,
 -0.011656252667307854,
 0.06292840093374252,
 -0.032876357436180115,
 -0.09122604876756668,
 -0.031175360083580017,
 0.0526994913816452,
 0.04703482612967491,
 -0.08420313149690628,
 -0.030056189745664597,
 -0.02074486576

In [22]:
print("Length", len(query_result))

Length 384


## Initializing the Pinecone

In [28]:
import os

In [29]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '79b14a1e-a399-4fd4-b79e-c8f8902cefd7')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [30]:
import pinecone
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "test" # put in the name of your pinecone index here


In [31]:
from langchain.vectorstores import Pinecone

In [32]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

## If you already have an index, you can load it like this

In [33]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
docsearch

## Similarity Search

In [34]:
query = "What is yolo?"

In [35]:
docs = docsearch.similarity_search(query, k=3)

In [36]:
docs

[Document(page_content='image each time leading to less false positives (has contextual information for detection) YOLO algorithm'),
 Document(page_content='CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than the other algorithms for image detection❖Pro: YOLO’s use of global information rather than only local information allows it to understand contextual information when doing object detection➢Does better in domains such as artwork due to this❖Con: YOLO lagged behind the SOTA models in object detection➢This is attributed to making many localization errors and unable to detect small object'),
 Document(page_content='CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy\n“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification')]

In [37]:
len(docs)

3

In [38]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00


In [39]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [40]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [41]:
from langchain.llms import OpenAI

In [42]:
llm = OpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [43]:
from langchain.chains import RetrievalQA

In [44]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [45]:
query = "What is yolo?"

In [46]:
print(qa.run(query))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



YOLO (You Only Look Once) is a fast and accurate algorithm for object detection in images. It uses global information to understand contextual information, making it suitable for domains such as artwork. However, it may lag behind other state-of-the-art models due to localization errors and its limited ability to detect small objects. YOLO also imposes spatial constraints on objects in images, which can limit its usefulness for irregular shapes.
